In [1]:
import torch
from torch import nn as nn
from models.nac import NAC
from models.nalu import NALU
from models.inalu import INALU
from utilities.train_utils import get_eval_loss, get_eval_preds
from utilities.train_utils import train
from torch.optim import RMSprop, Adam
import numpy as np
import pandas as pd

## Function to create Toy Dataset

In [2]:
def make_data(min_val, max_val, num_obs, op, operands):
    data = np.random.uniform(min_val, max_val, size=(num_obs, operands))
    if op == '+':
        targets = 0
        for i in range(operands):
            targets += data[:, i]
    elif op == '-':
        targets = 0
        for i in range(operands):
            targets -= data[:, i] 
    elif op == '*':
        targets = 1
        for i in range(operands):
            targets *= data[:, i] 
    elif op == '/':
        targets = data[:, 0]
        for i in range(1,operands):
            targets /= data[:, i] 
    elif op == 'permenant':
        targets = data[:, 0]*data[:, 3] + data[:, 1]*data[:,2]
    elif op == 'determenant':
        targets = data[:, 0]*data[:, 3] - data[:, 1]*data[:,2]
        data = np.random.uniform(min_val, max_val, size = (num_obs,1))
        targets = data ** 2
    elif op == 'sqrt':
        data = np.random.uniform(min_val, max_val, size = (num_obs, 1))
        targets = np.sqrt(data)
    return data, targets

## Function to compute accuracy of predictions

In [3]:
def accuracy_score(preds, targets, tol=1e-1):
    '''
    Computes prediction accuracy by checking if
    predictions are equal to the target upto `tol`
    places after decimal.
    '''
    preds = preds.cpu().numpy().flatten()
    targets = targets.cpu().numpy().flatten()
    accuracy = np.isclose(preds, targets, rtol=tol)
    accuracy = accuracy.astype(np.int32).mean()
    return accuracy * 100


## Using NAC to test addition and Subtraction

## Addition

In [4]:
X_train, Y_train = make_data(20, 40, 10000, '+', 2)
X_valid, Y_valid = make_data(20, 40, 2000, '+', 2)
X_test, Y_test = make_data(0, 60, 1000, '+', 2) # Test set contains both interpolated
                                                    # and extrapolated data
    
model = NAC(X_train.shape[1], 1)
        
model = train(model, nn.SmoothL1Loss(), RMSprop, X_train, Y_train,
                  X_valid, Y_valid, local_patience=15, patience=15, batch_size=16)


[Epoch: 5] Training loss after 2969 batches: 1.127:  75%|▋| 468/625 [00:00<00:00IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 15] Training loss after 8822 batches: 0.379:  11%| | 71/625 [00:00<00:00IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 24] Training loss after 14812 batches: 0.226:  70%|▋| 436/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

In [5]:
 test_preds, test_targets = get_eval_preds(model, X_test,Y_test, 16, False)
    
    
test_acc = accuracy_score(test_preds, test_targets)
    
print('Test accuracy for [{}]: {:.2f}%'.format('+', test_acc))

Test accuracy for [+]: 100.00%


In [6]:
output = model(torch.tensor([38,90]).float())
output

tensor([128.0000], grad_fn=<SqueezeBackward3>)

## Addition with multiple operands

In [7]:
accurac = []
for oper in range(10):
    X_train, Y_train = make_data(20, 40, 10000, '+', oper+1)
    X_valid, Y_valid = make_data(20, 40, 2000, '+', oper+1)
    X_test, Y_test = make_data(0, 60, 1000, '+', oper+1) # Test set contains both interpolated
                                                    # and extrapolated data
    
    model = NAC(X_train.shape[1], 1)
        
    model = train(model, nn.SmoothL1Loss(), RMSprop, X_train, Y_train,
                  X_valid, Y_valid, local_patience=15, patience=15, batch_size=16)
    test_preds, test_targets = get_eval_preds(model, X_test,Y_test, 16, False)
    test_acc = accuracy_score(test_preds, test_targets)
    s = 'Test accuracy for [{}] with {} operands: {:.2f}%'.format('+', oper+1,test_acc)
    accurac.append(s)

[Epoch: 5] Training loss after 2963 batches: 1.114:  74%|▋| 462/625 [00:00<00:00IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 12] Training loss after 7160 batches: 0.461:  45%|▍| 284/625 [00:00<00:0IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 19] Training loss after 11514 batches: 0.287:  42%|▍| 263/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

[Epoch: 43] Training loss after 26277 batches: 0.347:   4%| | 26/625 [00:00<00:0IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 45] Training loss after 27733 batches: 0.329:  37%|▎| 232/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 51] Training loss after 31495 batches: 0.290:  39%|▍| 244/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

[Epoch: 82] Training loss after 51161 batches: 0.239:  86%|▊| 535/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 91] Training loss after 56596 batches: 0.216:  55%|▌| 345/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 98] Training loss after 61247 batches: 0.200:  99%|▉| 621/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

[Epoch: 10] Training loss after 5674 batches: 1.767:   8%| | 48/625 [00:00<00:00IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 15] Training loss after 8881 batches: 1.129:  21%|▏| 130/625 [00:00<00:0IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 18] Training loss after 10829 batches: 0.926:  32%|▎| 203/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

[Epoch: 38] Training loss after 23451 batches: 1.356:  52%|▌| 325/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 45] Training loss after 27910 batches: 1.139:  65%|▋| 409/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 52] Training loss after 32416 batches: 0.981:  86%|▊| 540/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

[Epoch: 69] Training loss after 42861 batches: 0.708:  58%|▌| 360/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 77] Training loss after 48020 batches: 0.632:  83%|▊| 519/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 84] Training loss after 52378 batches: 0.579:  80%|▊| 502/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

[Epoch: 13] Training loss after 7557 batches: 2.506:   9%| | 56/625 [00:00<00:00IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 18] Training loss after 11157 batches: 1.697:  85%|▊| 531/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 30] Training loss after 18623 batches: 1.017:  80%|▊| 497/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

[Epoch: 58] Training loss after 36044 batches: 1.262:  67%|▋| 418/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 64] Training loss after 39868 batches: 1.141:  79%|▊| 492/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 71] Training loss after 43973 batches: 1.034:  36%|▎| 222/625 [00:00<00:IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order t

In [8]:
accurac

['Test accuracy for [+] with 1 operands: 100.00%',
 'Test accuracy for [+] with 2 operands: 100.00%',
 'Test accuracy for [+] with 3 operands: 100.00%',
 'Test accuracy for [+] with 4 operands: 100.00%',
 'Test accuracy for [+] with 5 operands: 100.00%',
 'Test accuracy for [+] with 6 operands: 100.00%',
 'Test accuracy for [+] with 7 operands: 100.00%',
 'Test accuracy for [+] with 8 operands: 100.00%',
 'Test accuracy for [+] with 9 operands: 100.00%',
 'Test accuracy for [+] with 10 operands: 100.00%']

In [9]:
output = model(torch.tensor([1,3900,1,1,2,1,1,156,1,1]).float())
output

tensor([4065.], grad_fn=<SqueezeBackward3>)

In [10]:
print(model)

NAC()


In [11]:
l = []
for parameters in model.parameters():
    print(parameters)
    l.append(parameters)

Parameter containing:
tensor([[9.0109, 9.0110, 9.0110, 9.0110, 9.0110, 9.0110, 9.0111, 9.0110, 9.0111,
         9.0110]], requires_grad=True)
Parameter containing:
tensor([[16.6356, 16.6355, 16.6356, 16.6355, 16.6355, 16.6355, 16.6355, 16.6356,
         16.6356, 16.6356]], requires_grad=True)


In [12]:
print(torch.tanh(l[0]))
print(torch.sigmoid(l[1]))

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], grad_fn=<TanhBackward0>)
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], grad_fn=<SigmoidBackward0>)


## Subtraction

In [ ]:
X_train, Y_train = make_data(20, 40, 10000, '-', 2)
X_valid, Y_valid = make_data(20, 40, 2000, '-', 2)
X_test, Y_test = make_data(0, 60, 1000, '-', 2) # Test set contains both interpolated
                                                    # and extrapolated data
    
model_ = NAC(X_train.shape[1], 1)
        
model_ = train(model_, nn.SmoothL1Loss(), RMSprop, X_train, Y_train,
                  X_valid, Y_valid, local_patience=15, patience=15, batch_size=16)

[Epoch: 5] Training loss after 2973 batches: 2.249:  76%|▊| 472/625 [00:00<00:00IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

[Epoch: 14] Training loss after 8157 batches: 0.820:   5%| | 31/625 [00:00<00:00

In [ ]:
 test_preds, test_targets = get_eval_preds(model_, X_test,Y_test, 16, False)
    
    
test_acc = accuracy_score(test_preds, test_targets)
    
print('Test accuracy for [{}]: {:.2f}%'.format('-', test_acc))

In [ ]:
output = model_(torch.tensor([38,90]).float())
output

## Subtraction with multiple operands

In [ ]:
accurac = []
for oper in range(10):
    X_train, Y_train = make_data(20, 40, 10000, '-', oper+1)
    X_valid, Y_valid = make_data(20, 40, 2000, '-', oper+1)
    X_test, Y_test = make_data(0, 60, 1000, '-', oper+1) # Test set contains both interpolated
                                                    # and extrapolated data
    
    model_ = NAC(X_train.shape[1], 1)
        
    model_ = train(model_, nn.SmoothL1Loss(), RMSprop, X_train, Y_train,
                  X_valid, Y_valid, local_patience=15, patience=15, batch_size=16)
    test_preds, test_targets = get_eval_preds(model_, X_test,Y_test, 16, False)
    test_acc = accuracy_score(test_preds, test_targets)
    s = 'Test accuracy for [{}] with {} operands: {:.2f}%'.format('-', oper+1,test_acc)
    accurac.append(s)

In [ ]:
accurac

In [ ]:
output = model_(torch.tensor([38,90,3,5,7,5,4,299,5,237]).float())
output

In [ ]:
l = []
for parameters in model_.parameters():
    print(parameters)
    l.append(parameters)

In [ ]:
print(torch.tanh(l[0]))
print(torch.sigmoid(l[1]))
print(torch.tanh(l[0])*torch.sigmoid(l[1]))

## Random function containing only addition and subtraction
## y = x1 + x2 - x3 + x4 - x5 + x6 -x7 - x8

In [ ]:
X_train = np.random.uniform(20, 40, size=(10000, 8))
Y_train = X_train[:,0] + X_train[:, 1] - X_train[:, 2] + X_train[:, 3] - X_train[:, 4] + X_train[:, 5] - X_train[:, 6] - X_train[:, 7]
X_valid = np.random.uniform(20, 40, size=(2000, 8))
Y_valid = X_valid[:,0] + X_valid[:, 1] - X_valid[:, 2] + X_valid[:, 3] - X_valid[:, 4] + X_valid[:, 5] - X_valid[:, 6] - X_valid[:, 7]
X_test = np.random.uniform(20, 40, size=(1000, 8))
Y_test = X_test[:,0] + X_test[:, 1] - X_test[:, 2] + X_test[:, 3] - X_test[:, 4] + X_test[:, 5] - X_test[:, 6] - X_test[:, 7]

In [ ]:
model1 = NAC(X_train.shape[1], 1)
        
model1 = train(model1, nn.SmoothL1Loss(), RMSprop, X_train, Y_train,
                  X_valid, Y_valid, local_patience=15, patience=15, batch_size=16)

In [ ]:
 test_preds, test_targets = get_eval_preds(model1, X_test,Y_test, 16, False)
    
    
test_acc = accuracy_score(test_preds, test_targets)
    
print('Test accuracy for function: {:.2f}%'.format(test_acc))

In [ ]:
output = model1(torch.tensor([1,2,3,4,5,6,7,8]).float())
output

In [ ]:
l = []
for parameters in model1.parameters():
    print(parameters)
    l.append(parameters)

In [ ]:
print(torch.tanh(l[0]))
print(torch.sigmoid(l[1]))
print(torch.tanh(l[0])*torch.sigmoid(l[1]))

## Using NALU to test arithmetic operations

In [ ]:
def make_data_(min_val, max_val, num_obs, op):
    data = np.random.uniform(min_val, max_val, size=(num_obs, 2))
    if op == '+':
        targets = data[:, 0] + data[:, 1]
    elif op == '-':
        targets = data[:, 0] - data[:, 1] 
    elif op == '*':
        targets = data[:, 0] * data[:, 1]
    elif op == '/':
        targets = data[:, 0] / data[:, 1]
    elif op == 'permenant':
        targets = data[:, 0] * ( data[:, 4] * data[:, 8] + data[:, 5] * data[:, 7]) + data[:, 1] * ( data[:, 3] * data[:, 8] + data[:, 5] * data[:, 6]) + data[:, 2] * ( data[:, 3] * data[:, 7] + data[:, 4] * data[:, 6])
    elif op == 'determenant':
        targets = data[:, 0] * ( data[:, 4] * data[:, 8] - data[:, 5] * data[:, 7]) - data[:, 1] * ( data[:, 3] * data[:, 8] - data[:, 5] * data[:, 6]) + data[:, 2] * ( data[:, 3] * data[:, 7] - data[:, 4] * data[:, 6])
    elif op == '^2':
        data = np.random.uniform(min_val, max_val, size = (num_obs,1))
        targets = data ** 2
    elif op == 'sqrt':
        data = np.random.uniform(min_val, max_val, size = (num_obs, 1))
        targets = np.sqrt(data)
    return data, targets

In [ ]:
eps = 1e-12
ops = ['+', '-', '*', '/', 'sqrt', '^2']
test_scores = {}

for op in ops:
    X_train, Y_train = make_data_(20, 40, 10000, op)
    X_valid, Y_valid = make_data_(20, 40, 2000, op)
    X_test, Y_test = make_data_(0, 60, 1000, op) # Test set contains both interpolated
                                                    # and extrapolated data
    
    if op == '^2':
        model3 = nn.Sequential(NALU(X_train.shape[1], 2), NALU(2, 1))
    else:
        model3 = NALU(X_train.shape[1], 1)
        
    model3 = train(model3, nn.SmoothL1Loss(), RMSprop, X_train, Y_train,
                  X_valid, Y_valid, patience=15, batch_size=16, num_epochs=250)
    
    test_preds, test_targets = get_eval_preds(model3, X_test,
                                              Y_test, 16, False)
    
    test_acc = accuracy_score(test_preds, test_targets)
    
    print('Test accuracy for [{}]: {:.2f}%'.format(op, test_acc))
    
    if op == 'sqrt':
        test_scores['sqrt(a)'] = test_acc
    elif op == '^2':
        test_scores['a ^ 2'] = test_acc
    else:
        test_scores['a '+op+' b'] = test_acc

## Using NALU to test arithmetic operations with 5 operands

In [ ]:
def make_data_1(min_val, max_val, num_obs, op):
    data = np.random.uniform(min_val, max_val, size=(num_obs, 5))
    if op == '+':
        targets = data[:, 0] + data[:, 1] + data[:, 2] + data[:, 3] + data[:, 4]
    elif op == '-':
        targets = data[:, 0] - data[:, 1] - data[:, 2] - data[:, 3] - data[:, 4]
    elif op == '*':
        targets = data[:, 0] * data[:, 1] * data[:, 2] * data[:, 3] * data[:, 4]
    elif op == '/':
        targets = data[:, 0] / data[:, 1] / data[:, 2] / data[:, 3] / data[:, 4]
    elif op == 'permenant':
        targets = data[:, 0] * ( data[:, 4] * data[:, 8] + data[:, 5] * data[:, 7]) + data[:, 1] * ( data[:, 3] * data[:, 8] + data[:, 5] * data[:, 6]) + data[:, 2] * ( data[:, 3] * data[:, 7] + data[:, 4] * data[:, 6])
    elif op == 'determenant':
        targets = data[:, 0] * ( data[:, 4] * data[:, 8] - data[:, 5] * data[:, 7]) - data[:, 1] * ( data[:, 3] * data[:, 8] - data[:, 5] * data[:, 6]) + data[:, 2] * ( data[:, 3] * data[:, 7] - data[:, 4] * data[:, 6])
    elif op == '^2':
        data = np.random.uniform(min_val, max_val, size = (num_obs,1))
        targets = data ** 2
    elif op == 'sqrt':
        data = np.random.uniform(min_val, max_val, size = (num_obs, 1))
        targets = np.sqrt(data)
    return data, targets

In [ ]:
eps = 1e-12
ops = ['+', '-', '*', '/']
test_scores = {}

for op in ops:
    X_train, Y_train = make_data_1(20, 40, 10000, op)
    X_valid, Y_valid = make_data_1(20, 40, 2000, op)
    X_test, Y_test = make_data_1(0, 60, 1000, op) # Test set contains both interpolated
                                                    # and extrapolated data
    
    if op == '^2':
        model2 = nn.Sequential(NALU(X_train.shape[1], 2), NALU(2, 1))
    else:
        model2 = NALU(X_train.shape[1], 1)
        
    model2 = train(model2, nn.SmoothL1Loss(), RMSprop, X_train, Y_train,
                  X_valid, Y_valid, patience=15, batch_size=16, num_epochs=250)
    
    test_preds, test_targets = get_eval_preds(model2, X_test,
                                              Y_test, 16, False)
    
    test_acc = accuracy_score(test_preds, test_targets)
    
    print('Test accuracy for [{}]: {:.2f}%'.format(op, test_acc))
    
    if op == 'sqrt':
        test_scores['sqrt(a)'] = test_acc
    elif op == '^2':
        test_scores['a ^ 2'] = test_acc
    else:
        test_scores['a '+op+' b'] = test_acc